Homework 4 - Recommendation systems and clustering everywhere

1. Recommendation system

1.1 Analyzing e fixing stuff

In [1]:
import pandas as pd
import numpy as np
import re
import time

In [3]:
tsv_path = "vodclickstream_uk_movies_03.csv"

Netflix = pd.read_csv(tsv_path)

In [5]:
Netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671736 entries, 0 to 671735
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    671736 non-null  int64  
 1   datetime      671736 non-null  object 
 2   duration      671736 non-null  float64
 3   title         671736 non-null  object 
 4   genres        671736 non-null  object 
 5   release_date  671736 non-null  object 
 6   movie_id      671736 non-null  object 
 7   user_id       671736 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 41.0+ MB


We want to change datetime in this way, we can work with this data

In [6]:
Netflix.datetime = pd.to_datetime(Netflix.datetime)
Netflix.release_date = pd.to_datetime(Netflix.release_date, errors='coerce')

We Will use an LSH algorithm and to use it we need shingles, so we will change he "genres" features in a way taìhat we can access to each genre 

In [7]:
Netflix.genres.apply(lambda row: row.split(','))
Netflix['genres_list'] = ''
Netflix['genres_list'] = Netflix.genres.apply(lambda row: [word.strip() for word in row.split(',')])

In [8]:
#We add the clicks feature to know how many time each user watched that film, in this way we don't lose the info of which film
#each user wathced a lot of times
Netflix['clicks'] = 1
user_clicks = Netflix.groupby(['user_id', 'title', 'genres']).size().reset_index(name='clicks')

# Sort the DataFrame by clicks in descending order for each user
user_clicks_sorted = user_clicks.sort_values(by=['user_id', 'clicks'], ascending=[True, False])

# Group by user_id and take the top 10 movies for each user
top_10_movies_per_user = user_clicks_sorted.groupby('user_id').head(10)

# Display the result (title, genres_list, and clicks of the top 10 movies for each user)
top_10_movies_per_user[['user_id', 'title', 'genres', 'clicks']].head()

,user_id,title,genres,clicks
0,00004e2862,Hannibal,"Crime, Drama, Thriller",1
6,000052a0a0,Looper,"Action, Drama, Sci-Fi, Thriller",9
3,000052a0a0,Frailty,"Crime, Drama, Thriller",3
5,000052a0a0,Jumanji,"Adventure, Comedy, Family, Fantasy",3
7,000052a0a0,Resident Evil,"Action, Horror, Sci-Fi",2


we want a matrix that remind which genre each user watched

In [9]:
all_genres = set(genre for genres_list in Netflix['genres_list'] for genre in genres_list)# avoid doubles
all_genres=list(all_genres)
all_user_ids = Netflix['user_id'].unique()
matrix2 = pd.DataFrame(0, index=all_genres, columns=all_user_ids)

# We put a 1 if the user saw a film of that type
for index, row in Netflix.iterrows():
    user_id = row['user_id']
    genres_list = row['genres_list']
    matrix2.loc[genres_list, user_id] = 1

#matrix2

1.2 Minhash Signatures

We want to simulate the permutation od the rows, to do it we use an hash function that we will be fixed after the start of the procedure. this is a procedure that we need to create a signature Matrix, also To reduce the number of calculation and comparison we want to create a signature matrix and divide this in band, we will use numpy to speed-up all the procedure

In [10]:
def hash_function(x,m,a,b):
  p=31
  c=((a*x+b)%p)
  return(c)


In [27]:
import random
import numpy as np
import pandas as pd

k = 19
num_rows = matrix2.shape[0]

#we choose the parameter of the hash formula
a = [random.randint(1, 100) for _ in range(k+1)]
b = [random.randint(1, 100) for _ in range(k+1)]
#initialization of the signature matrix
Signature_hash = np.full((k+1, len(all_user_ids)), 10000)
m=-1#temporary number that we need because the resulting matrix will be a numpy
for j in all_genres:
    m+=1
    s=-1 #same reason of m
    for h in all_user_ids:
        s+=1
        if matrix2.loc[j, h] == 1:

          for i in range(k+1):
            #we choosed to keep the min value, like in the paper that was proposed in the HW
            Signature_hash[i,s]=min(hash_function(m,113,a[i],b[i]),Signature_hash[i,s])



#Signature_hash
#to work again with the user_ids we want the matrix like a Dataframe
Signature_hash_df = pd.DataFrame(Signature_hash, index=range(k+1), columns=all_user_ids)

1.3 LSH Algorithm

The LSH is the combination of the following passages, now the important thing is in how many buckets we want to divide the users. We used this parameters for the buckets, bands because after we tried a lot we found that this was a good compromise for to obtain a good solution in a good amount of time

In [28]:
#Band
#we want to divide the band to reduce the number of comparison, we know that more rows will mean an higher
#precision but we will have an higher cost of calculation
rows_per_band = 5
num_bands = k // rows_per_band

buckets = {}

for band_index in range(num_bands+1):
  n = [random.randint(1, 100) for _ in range(rows_per_band+1)]

  for i in all_user_ids:
    hash_values=0
    # isolatung the column and the rows
    start_row = band_index * rows_per_band
    end_row = ((band_index + 1) * rows_per_band)-1
    v=Signature_hash_df.loc[start_row:end_row,i]
    #we hash again this value and we will use it like a key for a dictionary, in this
    #way we can divide the users, the users in the same bucket will be more similar
    for g in range(len(v)):
      hash_values+=v[g+band_index * rows_per_band]*n[g]
    hash_values=hash_values%41
    if hash_values not in buckets:
      buckets[hash_values] = []
    buckets[hash_values].append(i)

We will calculate the Similarity with the Jaccard Similarity,
so number of buckets were both users are present divided by the number of buckets where it is present alone

In [29]:
def similarity(user1, user2):

    counter=0#we need this to calculate in how many buckets that they are present
    key=buckets.keys()
    for s in key:
      if user1 in buckets[s] and user2 in buckets[s]:
        counter+=1
    similarity_value=counter/(num_bands+1)

    return similarity_value


To find the 2 user more similar to one we will use the function described before iterating it on all the users of the same buckets

In [30]:
def two_most_similar(user):
    v = []
    key=buckets.keys()
    for k in key:
        if user in buckets[k]:
            # We don't want to calculate the similarity with each other user, but only with the users of the same bucket
            for h in buckets[k]:
                if h != user:
                    sim = similarity(user, h)
                    v.append((h, sim))

    # Ordering with the higher similarity
    v=list(set(v))
    sorted_list = sorted(v, key=lambda x: x[1], reverse=True)

    # We need only the best 2
    best = sorted_list[:2]

    return best

In [31]:
def Raccomandation(user):
  #2 temporary variable
  clicks1=[]
  clicks2=[]

  best=two_most_similar(user)#finding the most 2 similar users to the one we are searching
  users,sim=zip(*best)
  users_list=list(users)
  #we take the film for each user
  movies_for_user1 = top_10_movies_per_user.loc[top_10_movies_per_user['user_id'] == users_list[0], 'title']
  movies_for_user2 = top_10_movies_per_user.loc[top_10_movies_per_user['user_id'] == users_list[1], 'title']
 #first we want the movies that they have in common
  tip=list(set(movies_for_user1).intersection(set(movies_for_user2)))

  if len(tip)<5: #if we haven't enough film

    temp_v = set(movies_for_user1) - set(tip) #we take the film that are diffirent for fist user
    temp_v = pd.Series(list(temp_v))
    #the following passages are to order in base of clicks
    for g in temp_v:
      clicks1.append(top_10_movies_per_user.loc[(top_10_movies_per_user['user_id'] == users_list[0]) & (top_10_movies_per_user['title'] == g),'clicks'].values)
    temp=zip(temp_v,clicks1)
    best_df = pd.DataFrame(temp, columns=['title', 'clicks'])
    best = best_df.sort_values(by='clicks', ascending=False).head(5)
    a=5-len(tip) #a indicates the film that i am missing
    if a<= len(best): #now we have enough films
      raccomandation=best[:a]
      titles=raccomandation['title'].tolist()
      final=tip+titles
    else: #we still haven't enough films
      titles=best['title'].tolist()
      final = list(tip) + list(titles)
       #we repeat the same process that we did for the first user to the second user
      temp_2 = list(set(movies_for_user2) - set(tip))

      for g in temp_2:
        clicks2.append(top_10_movies_per_user.loc[(top_10_movies_per_user['user_id'] == users_list[1]) & (top_10_movies_per_user['title'] == g),'clicks'].values)
      temp=zip(temp_2,clicks2)
      best_df = pd.DataFrame(temp, columns=['title', 'clicks'])
      best = best_df.sort_values(by='clicks', ascending=False)
      a=5-len(final)
      raccomandation=best[:a] 

      titles2=raccomandation['title'].tolist()
      final=final+titles2


  elif len(tip)>6: #we have too much film , so we order for the click and then take only the best one
    for h in tip:
      clicks.append(top_10_movies_per_user.loc[(top_10_movies_per_user['user_id'] == users_list[0]) & (top_10_movies_per_user['title'] == h), 'clicks']+top_10_movies_per_user.loc[(top_10_movies_per_user['user_id'] == users_list[1])& (top_10_movies_per_user['title'] == h), 'clicks'])
    temp=zip(tip,clicks)
    best_df = pd.DataFrame(temp, columns=['title', 'clicks'])
    best = best_df.sort_values(by='clicks', ascending=False).head(5)
    titles=best['title'].tolist()
    final = list(tip) + list(titles)

  return(final)


In [32]:
Query=Raccomandation('2428de4ee1')
Query

['Christmas Inheritance',
 'A Christmas Prince',
 'Fallen',
 'Budapest',
 'Deadly Scholars']

# 4. Command Line Question

The executable script file and the output screen can be found in the repository under the name CommandLine.sh and CommandLine.png. Below I put the text of the file for easier reading and understanding.

```bash
#!/bin/bash
input_file="vodclickstream_uk_movies_03.csv"

# 1. What is the most-watched Netflix title?
# group by the uniques titles and count the number of occurrences, saving them in a txt file
awk -F ',' '{print $4}' $input_file | sort | uniq -c | sort -nr > title_count.txt
# get the title from the txt file
most_watched_title=$(head -n 1 title_count.txt | awk '{$1=""; print $0}' | sed 's/^ *//')
echo "Most watched title: $most_watched_title"

# 2. The average time of subsequent clicks on Netflix.com
# filtering the duration tha has to be >= 0.0 and saving the values in a new file
awk -F ',' '{ if ($3 >= 0.0) print $0 }' $input_file > filtered_duration.csv
# the sum of duration (column 3) and the average of them
avg_duration=$(awk -F ',' '{print $3}' filtered_duration.csv | awk '{sum += $1} END {print sum/NR}')
# convert the duration from seconds to minutes
avg_min=$(awk -v avg_duration="$avg_duration" 'BEGIN { avg_min = avg_duration / 60; printf "%.2f\n", avg_min }')
# print the result
echo "Average duration of subsequent clicks: $avg_min minutes"

# 3. ID of the user that has spent the most time on Netflix
# group by the user_id and sum the duration time of each user, then sorting them and saving them in a txt file
awk -F ',' 'NR>1 { sum[$NF] += $3 } END { for (i in sum) print i, sum[i] }' filtered_duration.csv | sort -k2,2nr > sorted_users.txt
# get the user_id from the txt file
high_time_user=$(head -n 1 sorted_users.txt | awk '{print $1}')
echo "The user that has spent the most time on Netflix: $high_time_user"
```

1. To get the most watched Netflix title, we first grouped by each title in the dataset, then counted each time this appeared and saved the results in descending order in a new file, called ```title_count.txt```. At the end we extracted the first title of the file previusly created.
2. The average time of subsequent clicks on Netflix, we interpreted it as the average of all the duration times in the column 'duration' of the dataset, since it represents how much time the user stayed on the title, so each value corresponds to one click. Sais this, we filtered the dataset taking into account only the durations that are not negative, then summed all the remaining and took the average. Finally we converted the result from seconds to minutes.
3. The id of the user that has spent the most time on netflix has been extracted by first grouping the dataset by each user id, then summing the durations of each one and sorting them in descending order in a file, called ```sorted_users.txt```. At the end we extracted the first user in the file just created.

The following are the results

![CommandLine](CommandLine.png)

# 5. Algorithmic Question

He was given an initial personal score of when he enrolled ***S***, which changes every time he takes an exam.\
Every of the exams he has to take is assigned a mark ***p***.\
Once he has chosen an exam, his score becomes equal to the mark **S = p**, and at the same time, the scoring system changes:
- If he takes an "easy" exam (the score of the exam being less than his score), every other exam's mark is increased by
the quantity **$S-p$**.
- If he takes a "hard" exam (the score of the exam is greater than his score), every other exam's mark is decreased by
the quantity **$p-S$**.
- The exam score can't be negative, so if in any update a mark goes below zero it must be collapsed to zero.

He wants to know which is the highest score possible he could get.


In [1]:
import itertools

In [2]:
def FedericosScore(S, marks, p):

    copy_marks = marks.copy()                                             # T(n)

    if len(copy_marks) == 1:   # base case                                # T(1)
        return copy_marks[0]                                              

    if p <= S: # easy exam                                                # T(1)
        copy_marks.pop(0)                                                 # T(n)
        copy_marks = [x+(S-p) if x+(S-p)>=0 else 0 for x in copy_marks]   # T(n) 
        S = p                                                             # T(1)

    else: # hard exam                                                     # T(n)
        copy_marks.pop(0)
        copy_marks = [x-(p-S) if x-(p-S) >= 0 else 0 for x in copy_marks] #T(n) 
        S = p
        
    return FedericosScore(S, copy_marks, copy_marks[0])                   # T(n-1) 


In [4]:
# input 1
S = 8
marks = [5, 7, 1]

permutations = [list(x) for x in itertools.permutations(marks, len(marks))] # all the possible permutations of the exams O(n!)

scores = {}
for permu in permutations:
    scores[FedericosScore(S=S, marks=permu, p=permu[0])] = permu
max_s = max(scores.keys())
print(f'The highest score Federico can get is: {max_s} with this oder: {scores[max_s]}')

The highest score Federico can get is: 11 with this oder: [7, 1, 5]


In [5]:
# input 2
S = 25
marks = [18, 24, 21, 32, 27]

#FedericosScore(S, marks, marks[0])

permutations = [list(x) for x in itertools.permutations(marks, len(marks))] # all the possible permutations of the exams 

scores = {}
for permu in permutations:
    scores[FedericosScore(S=S, marks=permu, p=permu[0])] = permu
max_s = max(scores.keys())
print(f'The highest score Federico can get is: {max_s} with this order: {scores[max_s]}')

The highest score Federico can get is: 44 with this order: [27, 21, 32, 18, 24]


In [ ]:
# input 3
S = 30
marks = [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]

permutations = [list(x) for x in itertools.permutations(marks, len(marks))] # all the possible permutations of the exams 

# memory exceeded

scores = {}
for permu in permutations:
    scores[FedericosScore(S=S, marks=permu, p=permu[0])] = permu
max_s = max(scores.keys())
print(f'The highest score Federico can get is: {max_s} with this order: {scores[max_s]}')

The idea behind this initial solution is to have a function, ```FedrericosScore```, that takes in input:
- ```S```: the initial score of the student
- ```marks```: the list of the exams' marks
- ```p```: the first exam's mark

Then, since the student can choose the order of the esams he can take, we compute all the possible $n!$ permutations of the exams and give them in input, one by one, to the function.\
The result will be stored in a dictionary with keys the score and as vaulues the specific permutation that will make the student obstain that final score.\
Finally, we take the maximum score, out of the dictionary's keys, and its associated value; in this way the student will know in which order he has to take its exams in order to obtain the maximum score possible.

time complexity of the function:\
Assuming $n$ is the lenght of the ```marks``` list
- ```copy_marks``` has a time complexity of $T(n)$
- ```if len(copy_marks) == 1``` has a constant time complexity $T(1)$
- ```if p <= S:``` has a total time complexity of $T(n)$, since it has to pop the first element, shifting all the others, and in the list comprehension has to iterate on all the elements of the list.
- ```else:``` has a total time complexity of $T(n)$ for the same reason above.
- ```return FedericosScore``` has a time complexity of $T(n-1)$ in the worst case, since depends on the number of elements left in the list.

The total running time $T(n)$ can be expressed as:
\begin{align*}
T(n) =
\begin{cases} 
n+1 & \text{ if $n = 1$} \\
T(n-1)+5n+1 & \text{ if $n>1$} 
\end{cases}
\end{align*}
Generalizing we'll have a $T(n-i)+(5 \cdot i)n + i$, which at the end will result in $T(n) = 5n^2 - 3n + 2 \implies O(n^2)$.  

So the function has a quadratic time complexity of $O(n^2)$, then it has to be applied $n!$ times, for each one of the permutations of the list, so at the end the whole algorithm will have a complexity of $O(n^2 \cdot n!)$

This can't be sustainbable, since is highly memory-comsuming computing all the $n!$ permutations of a list as $n$ grows; for this reason this method doesn't work on the ```intput 3``` since has $n=12$ elements and all its permutations will be $479.001.600$. 

## Optimization

In [7]:
def Federicos_score_opt(S, marks):

    if len(marks) == 0:                                                                # T(1)     
        return S                                                                       # T(1)     
    
    maxScore = 0  # keep track of the maximum score Federico can achieve               # T(1)
    for i, mark in enumerate(marks):                                                   # T(n) 
        update_marks = marks[:i] + marks[i+1:] # don't consider the current exam       # T(n) 
        newScore = mark                                                                # T(1)
        if mark <= S: # esasy exam                                                     # T(1)
            update_marks = [x+(S-mark) if x+(S-mark)>=0 else 0 for x in update_marks]  # T(n)
        else: # hard exam
            update_marks = [x-(mark-S) if x-(mark-S)>=0 else 0 for x in update_marks]  # T(n)

        # comparing the current maxScore with the result of the recursive call.
        maxScore = max(maxScore, Federicos_score_opt(newScore, update_marks))          # T(n-1)
    return maxScore                                                                    # T(1)

In [8]:
# input 1
S_1 = 8
p_1 = [5, 7, 1]
result_1 = Federicos_score_opt(S_1, p_1)
print("Output 1:", result_1)

Output 1: 11


In [12]:
S_2 = 25
p_2 = [18, 24, 21, 32, 27]
result_2 = Federicos_score_opt(S_2, p_2)
print("Output 2:", result_2)

Output 2: 44


In [11]:
S_3 = 30
p_3 = [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]
result_3 = Federicos_score_opt(S_3, p_3)
print("Output 3:", result_3)

# took 16 mins

Output 3: 109


In this solution we don't have to explicitally compute all the possible permutations of the exams, because the optimized function explores all possible combinations of the exams recursivly and eventually finds the combination that guarantees the highest score possible by exploring different paths for each exam in the list. 

Time complexity of the function:\
Assuming $n$ is the lenght of the ```marks``` list
- ```if len(copy_marks) == 1``` has a constant time complexity $T(1)$
- ```for i, mark in enumerate(marks)``` has a linear time of $T(n)$ because we iterate through all the exams
- ```update_marks = marks[:i] + marks[i + 1:]``` has a linear time complexity $T(n)$ since we're performing a list slicing
- ```if mark <= S``` loop has a total complexity of $T(n)$ since the list comprehension inside explores all the elements in the list
- ```else``` loop has the complexity of $T(n)$ for the same reason above
- ```maxScore = max(maxScore, Federicos_score_opt(newScore, update_marks))``` has a time complexity of $T(n-1)$ in the worst case, since depends on the number of elements left in the list.

Since the recursive call of the function is inside the loop of complexity $O(n^2)$, this significantly contributes to the overall complexity of the function, that is $O(n^3)$.\
This is an improvement in respect to the previus algorithm, both on time complexity and memory complexity, since we're now able to actually see a result for the exam's list of the third input.

# ChatGPT optimization implementation 

In [31]:
def Federicos_score_chatGPT(S, marks, store={}):
    if len(marks) == 0:
        return S

    if (S, tuple(marks)) in store:
        return store[(S, tuple(marks))]

    maxScore = 0
    for i, mark in enumerate(marks):
        update_marks = marks[:i] + marks[i + 1:]
        newScore = mark
        if mark <= S:
            update_marks = [x + (S - mark) if x + (S - mark) >= 0 else 0 for x in update_marks]
        else:
            update_marks = [x - (mark - S) if x - (mark - S) >= 0 else 0 for x in update_marks]

        maxScore = max(maxScore, Federicos_score_chatGPT(newScore, update_marks, store))

    store[(S, tuple(marks))] = maxScore
    return maxScore


In [32]:
# input 1
S_1 = 8
p_1 = [5, 7, 1]
result_1 = Federicos_score_chatGPT(8, [5,7,1])
print("Output 1:", result_1)

Output 1: 11


In [29]:
S_2 = 25
p_2 = [18, 24, 21, 32, 27]
result_2 = Federicos_score_chatGPT(S_2, p_2)
print("Output 2:", result_2)

Output 2: 44


In [30]:
S_3 = 30
p_3 = [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]
result_3 = Federicos_score_chatGPT(S_3, p_3)
print("Output 3:", result_3)

# took 2.5 seconds

Output 3: 109


Optimization Approach:

- **Memoization**: Introduced a memoization dictionary (```store```) to store previously computed results for combinations of ```S``` and ```marks```. This prevents redundant calculations by checking if a specific combination has been encountered before. If so, it retrieves the result from memo instead of recomputing it.

- **Avoidance of Recomputation**: By storing and retrieving results from the ```store``` dictionary, the function bypasses the need to recompute solutions for the same combination of ```S``` and ```marks```. This significantly reduces the number of recursive calls and computations required, leading to a more efficient algorithm overall.

This optimization effectively reduces the time complexity of the algorithm from $O(n^3)$ to $O(n^2)$ by avoiding redundant computations through memoization.